# Import the required libraries ⏰

In [ ]:
import requests #Requests will allow you to send HTTP/1.1 requests using Python. With it, you can add content like headers, form data, multipart files
import string #For a string manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sequences
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding,GRU,Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer #This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) 
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam #to call the adam optimizer
from tensorflow.keras.models import Sequential # create a sequential models
#from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from tensorflow.keras.layers import SimpleRNN #to run the rnn model

#Data Loading ⛹

In [ ]:
data = open("/content/sample.txt").read() # data is being stored in text file so, no need to use pandas

In [ ]:
len(data) #Display the lenght

2427249

In [ ]:
print(data ) 

The Project Gutenberg eBook of The Republic, by Plato

This eBook isin a different spirit. for the use of anyone anywhere in the United States and

In what manner?

At present, I said, the students of philosophy are quite young;
beginning when they are hardly past childhood, they devote only the
time saved from moneymaking and housekeeping to such pursuits; and even
those of them who are reputed to have most of the philosophic spirit,
when they come within sight of the great difficulty of the subject, I
mean dialectic, take themselves off. In after life when invited by some
one else, they may, perhaps, go and hear a lecture, and about this they
make much ado, for philosophy is not considered by them to be their
proper business: at last, when they grow old, in most cases they are
extinguished more truly than Heracleitus’ sun, inasmuch as they never
light up again. (Heraclitus said that the sun was extinguished every
evening and relighted every morning.)

But what ought to be their cours

#Data preparation/preprocessing 👁

In [ ]:
# apply cleaning function
import string
def clean_text(data):
  data = data.replace('--', ' ')  #Replace ‘–‘ with a white space so we can split words better.
  tokens = data.split() #Split by Whitespace
  table = str.maketrans(' ', ' ', string.punctuation)#We can use the function maketrans() to create a mapping table
  tokens = [w.translate(table) for w in tokens]#Python offers a function called translate() that will map one set of characters to another.
  tokens = [word for word in tokens if word.isalpha()]#The isalpha() method returns True if all the characters are alphabet letters (a-z).
  return tokens

tokens = clean_text(data)
print(tokens[:50])


['The', 'Project', 'Gutenberg', 'eBook', 'of', 'The', 'Republic', 'by', 'Plato', 'This', 'eBook', 'isin', 'a', 'different', 'spirit', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'United', 'States', 'and', 'In', 'what', 'manner', 'At', 'present', 'I', 'said', 'the', 'students', 'of', 'philosophy', 'are', 'quite', 'young', 'beginning', 'when', 'they', 'are', 'hardly', 'past', 'childhood', 'they', 'devote', 'only']


In [ ]:
len(tokens)# calculate length of tokens

432747

In [ ]:
len(set(tokens))#total no words

11180

In [ ]:
length_sentence = 50 + 1 # we have input of length 50 and one preditected output
#We can organize the long list of tokens into sequences of 50 input words and 1 output word
lines = []
for i in range(length_sentence, len(tokens)):
  seq = tokens[i-length_sentence:i] 
  line = ' '.join(seq) # We will transform the tokens into space-separated strings for later storage in a file
  lines.append(line)
  if i > 160000:#limit our dataset to 200000 words.
    break

    
    

In [ ]:
print(len(lines))

159951


In [ ]:
lines[0]

'The Project Gutenberg eBook of The Republic by Plato This eBook isin a different spirit for the use of anyone anywhere in the United States and In what manner At present I said the students of philosophy are quite young beginning when they are hardly past childhood they devote only the'

In [ ]:
#texts_to_sequences() transforms each text in texts to a sequence of integers.
tokenizer = Tokenizer() #map each word in our vocabulary to a unique integer and encode our input sequences
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines) #use the fit Tokenizer to encode all of the training sequences, converting each sequence from a list of words to a list of integers.

In [ ]:
sequences = np.array(sequences)# convert sequence to numpy array

In [ ]:
sequences [:100]

array([[   1,  206,  541, ..., 3247,   48,    1],
       [ 206,  541, 1161, ...,   48,    1,  108],
       [ 541, 1161,    2, ...,    1,  108, 1802],
       ...,
       [4059,   19,   76, ..., 2323,   15,   16],
       [  19,   76,   38, ...,   15,   16,  132],
       [  76,   38,  442, ...,   16,  132,  162]])

In [ ]:
sequences = np.array(sequences)
x, y = sequences[:, :-1], sequences[:,-1]

In [ ]:
x[0]

array([   1,  206,  541, 1161,    2,    1,  169,   19,   63,   28, 1161,
       9326,    7,  267,  275,   26,    1,  154,    2, 2321, 2320,    6,
          1,  705,  185,    3,    6,   45,  209,   32,  280,   22,   35,
          1, 1613,    2,  122,   14,  274,  330,  406,   39,   16,   14,
        252,  670, 1454,   16, 3247,   48])

In [ ]:
#tokenizer.word_index gives the mapping of each unique word to its numerical equivalent.
#tokenizer.word_index gives the vocab_size.
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

9327

In [ ]:
y = to_categorical(y, num_classes=vocab_size)# apply one hot encoding
#Now that we have encoded the input sequences, we need to separate them into input (X) and output (y) elements.

In [ ]:
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
seq_length = x.shape[1]
seq_length

50

#  RNN model 🪆

In [ ]:
model = Sequential() # Create a sequential model
# Add Input Embedding Layer
model.add(Embedding(vocab_size, 50, input_length=seq_length))

model.add(SpatialDropout1D(0.2)) #create a dropout by 0.2
model.add(SimpleRNN(256, dropout=0.2)) #memory size 256


model.add(Dense(vocab_size, activation='softmax')) #create a dense function using softmax activation function

#LSTM model ▶ Trial01 ✅

In [ ]:
model = Sequential()
# Add Input Embedding Layer
model.add(Embedding(vocab_size, 50, input_length=seq_length))



model.add(LSTM(256))

model.add(Dense(512, activation='tanh'))

model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
history=model.fit(x, y, batch_size = 128, epochs = 100)

In [ ]:
seed_text=lines[100]
seed_text

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict(encoded, verbose=0)
    y_predict= np.argmax(y_predict, axis=1) 
    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)


In [ ]:
seed_text

##LSTM model ▶ Trial01 ✅
(lstm with more depth( add dense layer as it might improve accuracy but in our case it is not useful).

In [ ]:
model = Sequential()
# # Add Input Embedding Layer
model.add(Embedding(vocab_size, 50, input_length=seq_length))


model.add(LSTM(256))
model.add(Dense(512, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(vocab_size, activation='softmax'))



#GRU layer ♈

In [ ]:
model = Sequential()
# Add Input Embedding Layer
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(GRU(256))

model.add(Dense(512, activation='tanh'))

model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
history=model.fit(x, y, batch_size = 128, epochs = 100)

In [ ]:
seed_text=lines[100]# the orignal  text
seed_text

##apply function of generate text


 we are going to generate words given a set of input words so we need random text as it defined here as seed text  ,number of words that the model will predict  them in the future a model   that it learns and predict ,tokenizer to convert text to numbers as the model can predict  , and also we can  use  padding to get length . then the model can predict the probability for the next word using the
context of the last 100 words.

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict(encoded, verbose=0)
    y_predict= np.argmax(y_predict, sequencesaxis=1) 
    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)


In [ ]:
seed_text